# LCA and Mass Balance Calculation 

Some of the following code was adapted from the 2022 version of Massimo Pizzol's Advanced LCA course notes (https://github.com/massimopizzol/advanced-lca-notebooks).

In [1]:
import brightway2 as bw
import bw2data
import bw2analyzer
import pandas as pd
import numpy as np

from lci_to_bw2 import * # import all the functions of this module

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from scipy import stats



import openpyxl

In [2]:
mydb = pd.read_csv('balancedb_excel_w_ecoinvent.csv', header = 0, sep = ",") 
mydb.head()

,Activity database,Activity code,Activity name,Activity unit,Activity type,Exchange database,Exchange input,Exchange amount,Exchange unit,Exchange type,...,Variance - Pedigree,Variance - Pedigree (Relative,Variance - Pedigree (Scaled),Variance - Total,"CI/2wP, half range of confidence interval",sigma,sigma*,sigma* - basic,sigma* - relative,sigma* - scaled
0,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Concrete production,2316.0,kilogram,production,...,NaN,NaN,NaN,0.0000,1.000000,1.000000,0.000000,NaN,NaN,NaN
1,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Cement,176.0,kilogram,technosphere,...,0.0127,0.000965,0.00127,0.0327,1.435715,1.198213,0.180831,0.141421,0.144793,0.145842
2,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Water,176.0,kilogram,technosphere,...,0.0127,0.000965,0.00127,0.0327,1.435715,1.198213,0.180831,0.141421,0.144793,0.145842
3,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Fine Aggregate,725.0,kilogram,technosphere,...,0.0127,0.003976,0.00127,0.0327,1.435715,1.198213,0.180831,0.141421,0.154841,0.145842
4,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Course Aggregate,1169.0,kilogram,technosphere,...,0.0127,0.006410,0.01270,0.0327,1.435715,1.198213,0.180831,0.141421,0.162513,0.180831


In [3]:
mydb = mydb.drop('Notes', 1)  # remove the columns not needed
mydb['Exchange uncertainty type'] = mydb['Exchange uncertainty type'].fillna(0).astype(int) # uncertainty as integers
mydb.head()

C:\Users\em398\AppData\Local\Temp\ipykernel_10100\3263229204.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  mydb = mydb.drop('Notes', 1)  # remove the columns not needed


,Activity database,Activity code,Activity name,Activity unit,Activity type,Exchange database,Exchange input,Exchange amount,Exchange unit,Exchange type,...,Variance - Pedigree,Variance - Pedigree (Relative,Variance - Pedigree (Scaled),Variance - Total,"CI/2wP, half range of confidence interval",sigma,sigma*,sigma* - basic,sigma* - relative,sigma* - scaled
0,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Concrete production,2316.0,kilogram,production,...,NaN,NaN,NaN,0.0000,1.000000,1.000000,0.000000,NaN,NaN,NaN
1,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Cement,176.0,kilogram,technosphere,...,0.0127,0.000965,0.00127,0.0327,1.435715,1.198213,0.180831,0.141421,0.144793,0.145842
2,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Water,176.0,kilogram,technosphere,...,0.0127,0.000965,0.00127,0.0327,1.435715,1.198213,0.180831,0.141421,0.144793,0.145842
3,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Fine Aggregate,725.0,kilogram,technosphere,...,0.0127,0.003976,0.00127,0.0327,1.435715,1.198213,0.180831,0.141421,0.154841,0.145842
4,balancedb_extended,Concrete production,Concrete production,kilogram,process,balancedb_extended,Course Aggregate,1169.0,kilogram,technosphere,...,0.0127,0.006410,0.01270,0.0327,1.435715,1.198213,0.180831,0.141421,0.162513,0.180831


In [4]:
#Choose uncertainty characterisation test (based on ecoinvent uncertainty characterisation)
# 0 - Basic
# 1 - Basic + Additional 


uncertainty_Char = 3
#exc_scales = np.array([
#    [0.141421356, 0.141421356, 0.141421356, 0.141421356, 0.141421356, 0.141421356],
#    [0.180831413, 0.180831413, 0.180831413, 0.180831413, 0.180831413,0]
#])


#Choose test - Basic (0.02) + Additional 
# 0 - [0,0,0,0,0] NOT USED
# 1 - [1,1,1,1,1] data quality score (0)
# 2 - [2,2,2,2,2] data quality score (0.001525)
# 3 - [3,3,3,3,3] data quality score (0.0127)
# 4 - [4,4,4,4,4] data quality score (0.0586)
# 5 - [5,5,5,5,5] data quality score (0.21)

exc_scales = np.array([
    [0,0,0,0,0,0],
    [0.141421356,0.141421356,0.141421356,0.141421356,0.141421356,0],
    [0.146714008, 0.146714008,0.146714008,0.146714008,0.146714008,0],
    [0.180831413, 0.180831413, 0.180831413, 0.180831413, 0.180831413,0],
    [0.280356915, 0.280356915, 0.280356915, 0.280356915, 0.280356915, 0],
    [0.479583152, 0.479583152, 0.479583152, 0.479583152, 0.479583152, 0]
])




exc_scale = exc_scales[uncertainty_Char]
exc_scale

array([0.18083141, 0.18083141, 0.18083141, 0.18083141, 0.18083141,
       0.        ])

In [5]:
bw.projects.set_current('Balances_Cement')

In [6]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [7]:
#bw.databases.clear() # line to use in case you had already databases in the project space
bw.databases # lists all databases

Databases dictionary with 2 object(s):
	balancedb_extended
	biosphere3

In [8]:
# Delete specific databases
#del bw.databases['balancedb'] 


In [9]:

bw2_db = lci_to_bw2(mydb) # a function from the lci_to_bw2 module
bw2_db

{('balancedb_extended', 'Concrete production'): {'name': 'Concrete production',
  'unit': 'kilogram',
  'type': 'process',
  'exchanges': [{'input': ('balancedb_extended', 'Concrete production'),
    'amount': 2316.0,
    'unit': 'kilogram',
    'type': 'production',
    'uncertainty type': 0,
    'Variance - Total': 0.0,
    'CI/2wP, half range of confidence interval': 1.0,
    'sigma': 1.0,
    'sigma*': 0.0},
   {'input': ('balancedb_extended', 'Cement'),
    'amount': 176.0,
    'unit': 'kilogram',
    'type': 'technosphere',
    'uncertainty type': 2,
    'loc': 5.170483995,
    'scale': 0.180831413,
    'Variance - Basic': 0.02,
    'Variance - Pedigree': 0.0127,
    'Variance - Pedigree (Relative': 0.000965112,
    'Variance - Pedigree (Scaled)': 0.00127,
    'Variance - Total': 0.0327,
    'CI/2wP, half range of confidence interval': 1.435714775,
    'sigma': 1.198213159,
    'sigma*': 0.180831413,
    'sigma* - basic': 0.141421356,
    'sigma* - relative': 0.144793343,
    'si

In [10]:
t_db = bw.Database('balancedb_extended') # database name in the excel file is the same
# shut down all other notebooks using the same project
t_db.write(bw2_db)

Writing activities to SQLite3 database:
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/05/2024 21:35:17
  Finished: 03/05/2024 21:35:17
  Total time elapsed: 00:00:00
  CPU %: 104.20
  Memory %: 1.16


In [11]:

functional_unit = {t_db.get('Concrete production'): 2316}
lca = bw.LCA(functional_unit)  
lca.lci()
print(lca.inventory)

  (0, 5)	2.0299999415874486
  (0, 4)	3.5070000304840514
  (0, 3)	1.5224999457132073
  (0, 2)	0.18034720048308375
  (0, 1)	168.6080007553101


In [12]:

#Characterisation factor set as 1
myLCIAdata = [[('balancedb_extended', 'Carbon dioxide'), 1.0]
]
method_key = ('simplemethod', 'imaginaryendpoint', 'imaginarymidpoint')
my_method = bw.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register()
my_method.write(myLCIAdata)
my_method.load()


[[('balancedb_extended', 'Carbon dioxide'), 1.0]]

In [13]:
lca = bw.LCA(functional_unit, method_key)
lca.lci(factorize=True)
lca.lcia()
lca_df = lca.to_dataframe()
print('characterised_inventory\n', lca.characterized_inventory.toarray())

print('Score \n',lca.score)

det_LCA = lca.score



characterised_inventory
 [[  0.         168.60800076   0.1803472    1.52249995   3.50700003
    2.02999994]]
Score 
 175.84784787357788


This is the **static** or **deterministic** result of this LCA. 

In [14]:
lca_df

,Activity,Flow,Region,Amount
0,Fly-ash,Carbon dioxide,None,168.608001
1,Course Aggregate,Carbon dioxide,None,3.507000
2,Fine Aggregate,Carbon dioxide,None,2.030000
3,Water,Carbon dioxide,None,1.522500
4,Cement,Carbon dioxide,None,0.180347


In [15]:

#pd.DataFrame(lca.technosphere_matrix.toarray().getI())

In [16]:
#pd.DataFrame(lca.demand_array)

In [17]:
#pd.DataFrame(lca.supply_array)

# Characterise uncertainty 


In [18]:
ec = t_db.get('Concrete production')  


In [19]:
exc = list(ec.exchanges())
exc

[Exchange: 2316.0 kilogram 'Concrete production' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 176.0 kilogram 'Cement' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 176.0 kilogram 'Water' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 725.0 kilogram 'Fine Aggregate' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 1169.0 kilogram 'Course Aggregate' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>,
 Exchange: 70.0 kilogram 'Fly-ash' (kilogram, None, None) to 'Concrete production' (kilogram, None, None)>]

In [20]:
#ec.as_dict()

In [26]:
# Define log normal distribution
from stats_arrays import LognormalUncertainty
import numpy as np

index = 0
for exchange in exc:
    if exchange['input'] != exchange['output']:
        exchange['uncertainty type'] = LognormalUncertainty.id # this is an integer (not a float)
        exchange['loc'], exchange['scale'] = np.log(exchange['amount']), exc_scale[index] 

        if exchange['amount']<0:
            exchange['loc'] = np.log(-exchange['amount'])
            exchange['negative'] = True
        exchange.save() # important
        print(exchange.uncertainty)
        
index += 1
    
    



{'uncertainty type': 2, 'loc': 5.170483995038151, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 5.170483995038151, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 6.586171654854675, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 7.063903961472068, 'scale': 0.180831413}
{'uncertainty type': 2, 'loc': 4.248495242049359, 'scale': 0.180831413}


# MC simulation

From Adv LCA course notes:
Sources here: [example](http://stackoverflow.com/questions/38532146/obtaining-distribution-of-results-from-lcia), [another example](https://brightwaylca.org/examples/getting-started.html), and [source code](https://bitbucket.org/cmutel/brightway2-calc/src/662740694a8c70074105b5dca45b58651adb5eb5/bw2calc/monte_carlo.py?at=default&fileviewer=file-view-default)


In [22]:

amount = 2316
iterations = 1000
# This is the montecarlo simulation
mc = bw.MonteCarloLCA({ec: amount}, method_key) 


In [23]:
lca = bw.LCA({ec: amount}, method_key)
lca.lci()
lca.lcia()


det_bioMatrix = lca.biosphere_matrix.toarray()
det_techMatrix = lca.technosphere_matrix.toarray()


print('technosphere matrix\n',pd.DataFrame(det_techMatrix))
print('biosphere matrix\n',pd.DataFrame(det_bioMatrix))
print('supply array\n',pd.DataFrame(lca.supply_array))
print('characterised_inventory\n', pd.DataFrame(lca.characterized_inventory.toarray()))


det_LCA = lca.score
det_bioProcess = det_bioMatrix[:,0]

det_techProcess = det_techMatrix[:,0]
det_balance =  det_techProcess.sum()+ det_bioProcess.sum()

print('LCIA Score \n',lca.score)
print('Process Biosphere Column \n', det_bioProcess)

print('Process Technosphere Column \n',det_techProcess)
print('Mass Balance Percent \n', det_balance*100, ' %')


technosphere matrix
         0    1    2    3    4    5
0  2316.0  0.0  0.0  0.0  0.0  0.0
1  -176.0  1.0  0.0  0.0  0.0  0.0
2  -176.0  0.0  1.0  0.0  0.0  0.0
3  -725.0  0.0  0.0  1.0  0.0  0.0
4 -1169.0  0.0  0.0  0.0  1.0  0.0
5   -70.0  0.0  0.0  0.0  0.0  1.0
biosphere matrix
      0      1         2       3      4      5
0  0.0  0.958  0.001025  0.0021  0.003  0.029
supply array
         0
0     1.0
1   176.0
2   176.0
3   725.0
4  1169.0
5    70.0
characterised_inventory
      0           1         2       3      4     5
0  0.0  168.608001  0.180347  1.5225  3.507  2.03
LCIA Score 
 175.84784787357788
Process Biosphere Column 
 [0.]
Process Technosphere Column 
 [ 2316.  -176.  -176.  -725. -1169.   -70.]
Mass Balance Percent 
 0.0  %


In [24]:
pd.DataFrame(lca.characterized_inventory.toarray())

,0,1,2,3,4,5
0,0.0,168.608001,0.180347,1.5225,3.507,2.03


Redo Monte-Carlo

In [25]:
scores = []  # 1-dimensional array filled with zeros
diff = []  # 1-dimensional array filled with zeros
diff_bio = []
supply=[]
bio=[]
tech=[]
balance=[]
LCA_charCO2 = []
LCA_charCO2_Percent =[]
LCA_charEO = []
LCA_charEO_Percent =[]
cutOffList01 = []
cutOffList05 = []
names=['Product/Tech - Concrete production',
'Tech - Cement',
'Tech - Water',
'Tech - Fine aggregate',
'Tech - Coarse aggregate',
'Tech - Fly-ash'
]


for iteration in range(iterations):
    next(mc)
    mc_techProcess = mc.technosphere_matrix.toarray()[:,0]
    mc_bioProcess = mc.biosphere_matrix.toarray()[:,0]
    mc_LCAcharCO2 = mc.characterized_inventory.toarray()[0,:]
    mc_LCAcharCO2_Percent = (mc_LCAcharCO2 /mc.score)
    mc_balance = mc_techProcess.sum()+mc_bioProcess.sum()
    if np.abs(mc_balance) <= 0.01:
        cutOff01 = True
    else:
        cutOff01 = False
    if np.abs(mc_balance) <= 0.05:
        cutOff05 = True
    else:
        cutOff05 = False 
        
    LCA_charCO2.append(mc_LCAcharCO2.transpose())
    LCA_charCO2_Percent.append(mc_LCAcharCO2_Percent.transpose())
    scores.append(mc.score)
    supply.append(mc.supply_array)
    bio.append(mc.biosphere_matrix.toarray()[:,0])
    tech.append(mc_techProcess)
    balance.append(mc_balance) 

    cutOffList01.append(cutOff01)
    cutOffList05.append(cutOff05)
    

InvalidParamsError: Real, positive scale (sigma) values are required for lognormal uncertainties.

In [ ]:

#Names order
tech_df = pd.DataFrame(tech,columns=names[0:6])
LCA_charCO2_df = pd.DataFrame(LCA_charCO2,columns=names[0:6])
LCA_charCO2_percent_df = pd.DataFrame(LCA_charCO2_Percent,columns=names[0:6])
diff_Tech = tech_df-det_techProcess
diff_Tech_Rel = diff_Tech/det_techProcess
supply_df = pd.DataFrame(supply,columns=names[0:6])
supply_transpose = supply_df.transpose()
balance_df = pd.DataFrame({"Mass Balance": balance})
MassBal_diff_Abs = balance_df.rename(columns={"Mass Balance": "Diff Mass Bal"}) - det_balance
scores_df = pd.DataFrame({"LCIA Score":scores})
diff_Score = scores_df.rename(columns={"LCIA Score": "Diff Score"}) - det_LCA


diffAbs_df = pd.concat([
    diff_Tech],axis=1)
diffRel_df = pd.concat([
    diff_Tech_Rel],axis=1)

diffAbs_df["Type"] = "Abs diff"
diffRel_df["Type"] = "Rel diff"
LCA_charCO2_df["Type"] = "Process CO2 LCIA absolute"
LCA_charCO2_percent_df["Type"] = "Process CO2 LCIA contribution"
LCA_charCO2_df = LCA_charCO2_df.rename_axis('Iteration')
LCA_charCO2_percent_df = LCA_charCO2_percent_df.rename_axis('Iteration')
diffAbs_df = diffAbs_df.rename_axis('Iteration')
diffRel_df = diffRel_df.rename_axis('Iteration')

quantities_df = pd.concat([
tech_df
],axis=1)

quantities_df = quantities_df.rename_axis('Iteration')


#balance
MCresults_df = pd.concat([
    scores_df, 
    balance_df,
    MassBal_diff_Abs,
    diff_Score
    ], axis=1)


MCresults_df = MCresults_df.rename_axis('Iteration')

quantities_df["Type"] = "Quantity (kg)"

quantities_df=pd.concat([
    quantities_df,
    diffAbs_df,
    diffRel_df,
    LCA_charCO2_df,
    LCA_charCO2_percent_df,
    ], axis=0)




MCresults_df.round(3)

In [ ]:
#MCresults_df

In [ ]:
#quantities_df




# Dataset manipulation


In [ ]:
MC_results_df = quantities_df.rename_axis('Type',axis='columns')

MCresults_df_long = pd.melt(MCresults_df,
                  var_name='Type', value_name='Value',ignore_index=False)

MCresults_df_long = MCresults_df_long.reset_index(drop=False)
MCresults_df_long['Exchanges'] = "Total - Process Level" 

#MCresults_df_long

In [ ]:

quantities_df_long = pd.melt(quantities_df, id_vars=['Type'],value_vars=names,
                  var_name='Exchanges', value_name='Value',ignore_index=False)

quantities_df_long = quantities_df_long.reset_index(drop=False)


#quantities_df_long

In [ ]:
combined_df_long = pd.concat([quantities_df_long, MCresults_df_long], axis=0)

#combined_df_long

# Export results to Excel

In [ ]:

name =["B","B + A"]


In [ ]:

quantities_df["Test"] = uncertainty_Char
MCresults_df["Test"] = uncertainty_Char
combined_df_long["Test"] = uncertainty_Char
quantities_df["Uncertainty Scenario"] = name[uncertainty_Char]
MCresults_df["Uncertainty Scenario"] = name[uncertainty_Char]
combined_df_long["Uncertainty Scenario"] = name[uncertainty_Char]

quantities_df_store = quantities_df
MCresults_df_store = MCresults_df
combined_df_long_store = combined_df_long

%store quantities_df_store
%store MCresults_df_store
%store combined_df_long_store


In [ ]:
filename='quantitiesMCsimulation'+str(uncertainty_Char)+'.xlsx'
excel_file=filename
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')  


#Export quantities dataframe to excel
quantities_df.to_excel(writer,sheet_name='QuantitiesSample',index=False) # to save it
MCresults_df.to_excel(writer,sheet_name='MCresults',index=False) # to save it
combined_df_long.to_excel(writer,sheet_name='Combined_Long',index=False) # to save it


# save the workbook to a file
writer.save()


print(f"DataFrames exported to {excel_file}")